In [24]:
import great_expectations as ge
import pandas as pd
from pathlib import Path

In [28]:
# Define the paths to the CSV files for each state
csv_paths = [
    Path("Resources/Alabama/Consumer Spending.csv"),
    Path("Resources/Alabama/Socioeconomic Characteristics.csv"),
    Path("Resources/Arkansas/Consumer Spending.csv"),
    Path("Resources/Arkansas/Socioeconomic Characteristics.csv"),
    Path("Resources/District of Columbia/Consumer Spending.csv"),
    Path("Resources/District of Columbia/Socioeconomic Characteristics.csv"),
    Path("Resources/Louisiana/Consumer Spending.csv"),
    Path("Resources/Louisiana/Socioeconomic Characteristics.csv"),
    Path("Resources/Maryland/Consumer Spending.csv"),
    Path("Resources/Maryland/Socioeconomic Characteristics.csv"),
    Path("Resources/Massachusetts/Consumer Spending.csv"),
    Path("Resources/Massachusetts/Socioeconomic Characteristics.csv"),
    Path("Resources/Mississippi/Consumer Spending.csv"),
    Path("Resources/Mississippi/Socioeconomic Characteristics.csv"),
    Path("Resources/New Jersey/Consumer Spending.csv"),
    Path("Resources/New Jersey/Socioeconomic Characteristics.csv"),
    Path("Resources/Washington/Consumer Spending.csv"),
    Path("Resources/Washington/Socioeconomic Characteristics.csv"),
    Path("Resources/West Virginia/Consumer Spending.csv"),
    Path("Resources/West Virginia/Socioeconomic Characteristics.csv"),]


In [29]:
# Create an empty list to store DataFrames
dataframes = []

In [30]:
# Loop through each CSV file path
for path in csv_paths:
    
    # Read the CSV file
    df = pd.read_csv(path)
    
    # Filter the DataFrame to keep only the relevant rows
    filtered_df = df[df['Name'].isin([
        'Median household income',
        'Average income',
        'Consumer expenditures per household on Dining out (Food away from home)', 
        'Consumer expenditures per household on Entertainment / Recreation',])].copy()
    
    # Add a "State" column
    filtered_df.loc[:, 'State'] = path.parts[-2]
    
    # Append the filtered DataFrame to the list
    dataframes.append(filtered_df)


In [31]:
# Combine all filtered DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

In [32]:
# Pivot the DataFrame to change expenditures from rows to columns
final_df = combined_df.pivot(index='State', columns='Name', values='Value')

In [33]:
# Reset the index to make 'State' a column
final_df.reset_index(inplace=True)

In [34]:
# Rename the columns 
final_df.rename(columns={
    'Consumer expenditures per household on Dining out (Food away from home)': 'Dining Out (per household)',
    'Consumer expenditures per household on Entertainment / Recreation' : 'Entertainment/Recreation (per household)',
    'Median household income' : 'Median Household Income',
    'Average income' :'Average Household Income',
}, inplace=True)


In [35]:
# Add $ to the values in each of the columns except 'State'
monetary_columns = ['Dining Out (per household)', 'Entertainment/Recreation (per household)', 'Median Household Income', 'Average Household Income']
for col in monetary_columns:
    if col in final_df.columns:
        final_df[col] = final_df[col].apply(lambda x: f"${float(x):,.2f}" if pd.notnull(x) else x)

In [36]:
# Define the paths to the City Population CSV file
population_df = pd.read_csv('Resources/max_population_cities.csv')

In [37]:
#Split the Dataframe into two Dataframes
# Define the states for each group
top_states = ['District of Columbia', 'Massachusetts', 'New Jersey', 'Maryland', 'Washington']
bottom_states = ['Alabama', 'Arkansas', 'West Virginia', 'Louisiana', 'Mississippi']

# Split the DataFrame into two separate DataFrames
Top_df = final_df[final_df['State'].isin(top_states)]
Bottom_df = final_df[final_df['State'].isin(bottom_states)]

In [38]:
Top_df

Name,State,Average Household Income,Dining Out (per household),Entertainment/Recreation (per household),Median Household Income
2,District of Columbia,"$150,292.00","$5,660.82","$5,190.11","$101,722.00"
4,Maryland,"$129,642.00","$4,629.46","$4,636.97","$98,461.00"
5,Massachusetts,"$134,568.00","$4,734.14","$4,722.96","$96,505.00"
7,New Jersey,"$135,170.00","$4,723.62","$4,694.21","$97,126.00"
8,Washington,"$122,880.00","$4,470.93","$4,509.82","$90,325.00"


In [39]:
Bottom_df

Name,State,Average Household Income,Dining Out (per household),Entertainment/Recreation (per household),Median Household Income
0,Alabama,"$82,992.00","$2,917.60","$3,050.64","$59,609.00"
1,Arkansas,"$79,592.00","$2,822.33","$2,973.20","$56,335.00"
3,Louisiana,"$83,169.00","$2,903.64","$2,959.73","$57,852.00"
6,Mississippi,"$74,149.00","$2,576.02","$2,678.09","$52,985.00"
9,West Virginia,"$75,575.00","$2,726.49","$2,964.69","$55,217.00"


In [40]:
# Group by state and sum the population if there are multiple entries for the same state
state_population = population_df.groupby('State')['avg_population'].sum().reset_index()

In [41]:
# Merge the population data with final_df
final_top_df = pd.merge(Top_df, state_population, on='State', how='left')

# Rename the columns 
final_top_df.rename(columns={'avg_population':'Population',}, inplace=True)


final_top_df

,State,Average Household Income,Dining Out (per household),Entertainment/Recreation (per household),Median Household Income,Population
0,District of Columbia,"$150,292.00","$5,660.82","$5,190.11","$101,722.00",672449.0
1,Maryland,"$129,642.00","$4,629.46","$4,636.97","$98,461.00",573520.0
2,Massachusetts,"$134,568.00","$4,734.14","$4,722.96","$96,505.00",659956.0
3,New Jersey,"$135,170.00","$4,723.62","$4,694.21","$97,126.00",306881.0
4,Washington,"$122,880.00","$4,470.93","$4,509.82","$90,325.00",744134.0


In [42]:
final_bottom_df=pd.merge(Bottom_df, state_population, on='State', how='left')

# Rename the columns 
final_bottom_df.rename(columns={'avg_population':'Population',}, inplace=True)

final_bottom_df

,State,Average Household Income,Dining Out (per household),Entertainment/Recreation (per household),Median Household Income,Population
0,Alabama,"$82,992.00","$2,917.60","$3,050.64","$59,609.00",220695.0
1,Arkansas,"$79,592.00","$2,822.33","$2,973.20","$56,335.00",202970.0
2,Louisiana,"$83,169.00","$2,903.64","$2,959.73","$57,852.00",373660.0
3,Mississippi,"$74,149.00","$2,576.02","$2,678.09","$52,985.00",148072.0
4,West Virginia,"$75,575.00","$2,726.49","$2,964.69","$55,217.00",47660.0


In [43]:
# Save the two DataFrames to CSV files
final_top_df.to_csv('Top5.csv', index=False)
final_bottom_df.to_csv('Bottom5.csv', index=False)